In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import torch

# Leer los datos utilizando Pandas
data = pd.read_csv('train-data.csv')
print(data)

# Eliminar la columna 'N'
data = data.drop("N", axis=1)
print(data)

# Información sobre el DataFrame
data.info()


In [ ]:
# Seleccionar y eliminar las columnas no numéricas
columnas_categoricas = data.select_dtypes(include=['object']).columns
columnas_fechas = data.select_dtypes(include=['datetime64']).columns
for columna in columnas_categoricas:
    le = LabelEncoder()
    data[columna] = le.fit_transform(data[columna])

# Información actualizada sobre el DataFrame
data.info()


In [ ]:
# Seleccionar las columnas que se utilizarán como características (X) y la columna objetivo (y)
X = data.iloc[:, [6, 7, 8]]
y = data.iloc[:, 11]

# Convertir los datos de Pandas a tensores de PyTorch
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Obtener el número de filas (muestras)
m = y.size(0)

# Función para normalizar las características
def featureNormalize(X):
    mu = torch.mean(X, dim=0)
    sigma = torch.std(X, dim=0)
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

# Normalizar las características
X_norm, mu, sigma = featureNormalize(X)

# Agregar una columna de unos para el término de sesgo (intercept)
X_norm = torch.cat((torch.ones((m, 1)), X_norm), dim=1)


In [ ]:
# Función para calcular la función de costo
def computeCostMulti(X, y, theta):
    m = y.size(0)
    h = torch.mm(X, theta)
    J = (1 / (2 * m)) * torch.sum((h - y) ** 2)
    return J

# Función para el descenso del gradiente
def gradientDescentMulti(X, y, theta, alpha, num_iters):
    m = y.size(0)
    J_history = []

    for i in range(num_iters):
        h = torch.mm(X, theta)
        theta -= (alpha / m) * torch.mm(X.t(), (h - y))
        J_history.append(computeCostMulti(X, y, theta))

    return theta, J_history


In [ ]:
# Hiperparámetros para el descenso del gradiente
alpha = 0.0003
num_iters = 20000

# Inicializar los parámetros theta a ceros
theta = torch.zeros((X_norm.size(1), 1), dtype=torch.float32)

# Realizar el descenso del gradiente
theta, J_history = gradientDescentMulti(X_norm, y, theta, alpha, num_iters)


In [ ]:
# Gráfica de la convergencia del costo a través de las iteraciones
plt.plot(range(len(J_history)), J_history, lw=2)
plt.xlabel('Número de iteraciones')
plt.ylabel('Costo J')
plt.show()

print('Theta calculado por el descenso del gradiente:', theta)


In [ ]:
# Datos de prueba para predecir el precio
X_array = [1.0, 403.0, 144.0, 260.0]

# Normalizar las características de prueba usando mu y sigma calculados previamente
X_array[1:4] = (X_array[1:4] - mu.numpy()) / sigma.numpy()

# Agregar el término de sesgo
X_array = [1.0] + X_array

# Convertir a tensor de PyTorch
X_array = torch.tensor(X_array, dtype=torch.float32)

# Calcular el precio estimado utilizando el modelo entrenado
price = torch.mm(X_array.view(1, -1), theta)
print('El precio estimado es de: ${:.0f}'.format(price.item()))
